In [213]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
import cv2
import numpy as np


In [214]:
# 超参数
EPOCH = 10  # 训练整批数据的次数

LR = 0.001  # 学习率
DOWNLOAD_MNIST = True  # 表示还没有下载数据集，如果数据集下载好了就写False
torch.manual_seed(1)

In [215]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [216]:
# Adjust the model to get a higher performance
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()# batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        #in_channels=1,out_channels=8,kernel_size=3,stride=1
        self.conv1 = nn.Conv2d(1, 8, 3, 1)# 输出数据大小变为28-3+1=26.所以batchx1x28x28 -> batchx8x26x26   
        self.conv2 = nn.Conv2d(8, 16, 3, 1)#第一个卷积层的输出通道数等于第二个卷积层的输入通道数。
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(2304, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        ptint(x.shape)
        x = F.relu(x)#（激活函数ReLU不改变形状）
        x = self.conv2(x)  
        x = F.relu(x)#（激活函数ReLU不改变形状）
        x = F.max_pool2d(x, 2)# batch*8x26x26  -> batch*8*13*13（2*2的池化层会减半，步长为2）此时输出数据大小变为13-3+2=12（卷积核大小为3），所以 batchx8x13x13 -> batchx16x12x12。
        x = torch.flatten(x, 1)
        #x = x.view(-1, 16*12*12)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [217]:
def dataloader():
    # Normalize the input (black and white image)
    transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3,1,1)),
     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
     ])   
    
    # Make train dataset split
    train_data = datasets.MNIST("mnist-data", train=True, download=True,
                       transform=transform)
    # Make test dataset split
    test_data = datasets.MNIST("mnist-data", train=False,
                       transform=transform)
    
    
    return train_data,test_data

In [218]:
def get_text_data(test_data):
    # 进行测试
    testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)
    dataiter = iter(testloader)
    images, labels = dataiter.next()
    classes = ('0', '1', '2', '3','4', '5', '6', '7', '8', '9')
    return images,labels,classes


In [219]:
def get_cnn_net():
    #返回不同的模型
    net = Net()                            
    #net = torchvision.models.vgg11()      
    #net = torchvision.models.vgg11(pretrained=True)    
    #net = torchvision.models.googlenet() 
    #net = torchvision.models.resnet18()   
    #net = torchvision.models.resnet18(pretrained=True)   
    #for param in net.parameters():#nn.Module有成员函数parameters()
        #param.requires_grad = False
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
    #net.fc = nn.Linear(512, 1000)#resnet18中有self.fc，作为前向过程的最后一层。
    #net = torchvision.models.resnet50()  
    #net = torchvision.models.resnet50(pretrained=True)   
    #print(net)
    return net


In [220]:
def train(train_data):
    # 批训练 50个samples
    # Torch中的DataLoader是用来包装数据的工具，它能帮我们有效迭代数据，这样就可以进行批训练
    train_loader = torch.utils.data.DataLoader(
        dataset=train_data,
        batch_size=BATCH_SIZE,
        shuffle=True  # 是否打乱数据，一般都打乱
    )   
    
        
    # 获取 cnn 网络
    net = get_cnn_net()
    # 模型加载到gpu中
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = net.to(device)

    # 优化器选择Adam
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)
    
    # 仅更新模型的fc层，默认情况下更新所有的参数
    #optimizer = torch.optim.Adam(net.fc.parameters(), lr=LR)
    
    # 定义损失函数
    loss_func = nn.CrossEntropyLoss()  # 目标标签是one-hotted
    # 把x和y 都放入Variable中，然后放入cnn中计算output，最后再计算误差
    # 开始训练
    for epoch in range(EPOCH):
        running_loss = 0.0
        for step, (inputs, labels) in enumerate(train_loader):  # 分配batch data
            # 数据加载到gpu中
            inputs = inputs.to(device)
            labels = labels.to(device)
            output = net(inputs)  # 先将数据放到cnn中计算output
            ### 梯度下降算法 ###
            loss = loss_func(output, labels)  # 损失函数，输出和真实标签的loss，二者位置不可颠倒
            optimizer.zero_grad()  # 清除之前学到的梯度的参数
            loss.backward()  # 反向传播，计算梯度
            optimizer.step()  # 应用梯度（权重更新）
            ### 梯度下降算法 ###
            # 数据加载到cpu中
            loss = loss.to('cpu')
            running_loss += loss.item()         
            if step % 50 == 0:   
                print('[%d, %5d] loss: %.3f' % (epoch + 1, step + 1, running_loss / 50)) 
                running_loss = 0.0    
    #保存模型
    torch.save(net.state_dict(), 'test_cifar_gpu.pkl')



In [221]:
def imshow(img):
    img = img /2 + 0.5    # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [222]:
def predict(test_data):
    # 获取测试数据集
    #images,labels,classes= get_text_data(test_data)
    # print images
    #imshow(torchvision.utils.make_grid(images))
    #print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
    
    testloader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False, num_workers=2)
    # 获取cnn网络
    net = get_cnn_net()
    # 加载模型
    net.load_state_dict(torch.load('test_cifar_gpu.pkl'))
    # 设置为推理模式
    net.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # 模型加载到gpu中
    net = net.to(device)
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # 数据加载到gpu中
            images = images.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # 数据加载回cpu
            outputs = outputs.to('cpu')
            labels  = labels.to('cpu')
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    

In [ ]:
if __name__ == "__main__":
    # 数据加载
    train_data,test_data = dataloader()

    # 训练（先训练再预测）
    train(train_data)
    # 预测（预测前将训练注释掉，否则会再训练一遍）
    predict(test_data)